In [1]:
import time
import random
import linecache
import math
import numpy as np
import tensorflow as tf

from nltk.corpus import BNCCorpusReader
from spacy.en import English
from collections import defaultdict, Counter
from six.moves import xrange

### ONE-TIME RUNS: PREPROCESSING

In [33]:
# %%time

# # parse bnc, write NOUN's as targets, VERB's as contexts in save_path

# bnc_path = '/home/jacobsuwang/Documents/UTA2017/BAYES-DL/WORD2VEC/DATA/bnc/'
# save_path = '/home/jacobsuwang/Documents/UTA2017/BAYES-DL/WORD2VEC/DATA/bnc_word_dep.txt'
# bnc = BNCCorpusReader(root=bnc_path, fileids=r'\w*.xml')

# print '... loading BNC'
# bnc_sents = [unicode(' '.join(sent).encode('ascii','ignore')) for sent in bnc.sents()]
# parser = English()
# count = 0
# start = time.time()
# with open(save_path,'a') as f:
#     for sent in parser.pipe(bnc_sents, n_threads=32, batch_size=10000):
#         count += 1
#         for token in sent:
#             if token.pos_=='NOUN':
#                 f.write(token.lemma_+'\t'+token.head.lemma_+'/'+token.dep_+'\n')
#             if token.head.pos_=='NOUN':
#                 f.write(token.head.lemma_+'\t'+token.lemma_+'/'+token.dep_+'-1\n')
#         if count%10000==0:
#             end = time.time()
#             print '   ...processed',count,'sentences (time elapsed: '+str(end-start)+' secs)'
#             start = end
#     print 'DONE! (in total',count,'sentences)'

In [ ]:
'''
DONE! (in total 6019197 sentences)
CPU times: user 1h 30min 29s, sys: 16.9 s, total: 1h 30min 46s
Wall time: 38m
'''

In [32]:
# %%time

# # count NOUN's and VERB's

# vr2count = Counter()
# cpt2count = Counter()

# count = 0
# start = time.time()
# with open(save_path,'r') as f:
#     for line in f:
#         count += 1
#         line = line.split('\t')
#         if len(line)!=2: continue
#         cpt,vr = line
#         vr2count[vr] += 1
#         cpt2count[cpt] += 1
#         if count%100000==0:
#             end = time.time()
#             print '   ...counted',count,'entries (time elapsed: '+str(end-start)+' secs)'
#             start = end
#     print 'DONE! (in total',count,'entries)'

In [ ]:
'''
DONE! (in total 54930729 entries)
CPU times: user 46 s, sys: 604 ms, total: 46.6 s
Wall time: 45.7 s
'''

In [31]:
# %%time

# # encode targets and contexts

# cpt2i = {}
# vr2i = {}
# def get_id(item, dic):
#     if item not in dic:
#         dic[item] = len(dic)
#     return dic[item]

# count = 0
# start = time.time()
# cutoff = 10
# bnc_word2vec = '/home/jacobsuwang/Documents/UTA2017/BAYES-DL/WORD2VEC/DATA/bnc_word2vec_dep.txt'
# with open(save_path,'r') as source, open(bnc_word2vec,'a') as target:   
#     for line in source:
#         count += 1
#         line = line.split('\t')
#         if len(line)!=2: continue
#         cpt,vr = line
#         if cpt2count[cpt]<cutoff or vr2count[vr]<cutoff: continue
#         cpt_id, vr_id = get_id(cpt,cpt2i), get_id(vr,vr2i)
#         target.write(str(cpt_id)+'\t'+str(vr_id)+'\n')
#         if count%100000==0:
#             end = time.time()
#             print '   ...written',count,'entries (time elapsed: '+str(end-start)+' secs)'
#             start = end
#     print 'DONE! (in total',count,'entries)'

In [ ]:
'''
DONE! (in total 54930729 entries)
CPU times: user 1min 15s, sys: 588 ms, total: 1min 15s
Wall time: 1min 15s
'''

In [26]:
# save trackers

# import cPickle
# stats_path = '/home/jacobsuwang/Documents/UTA2017/BAYES-DL/WORD2VEC/DATA/bnc_word2vec_stats.p'
# cPickle.dump((count,cpt2i,vr2i), open(stats_path,'w'))

### WORD2VEC (SKIP-GRAM, DEPENDENCY)

In [2]:
import cPickle

bnc_word2vec = '/home/jacobsuwang/Documents/UTA2017/BAYES-DL/WORD2VEC/DATA/bnc_word2vec_dep.txt'
stats_path = '/home/jacobsuwang/Documents/UTA2017/BAYES-DL/WORD2VEC/DATA/bnc_word2vec_stats.p'
count,cpt2i,vr2i = cPickle.load(open(stats_path,'r'))

In [3]:
linecache.clearcache()

data_ids = range(count)
random.shuffle(data_ids)

class DataIterator:
    
    def __init__(self, data_ids):
        self.data_ids = data_ids
        self.epoch = 0
        self.cursor = 0
        self.size = len(data_ids)
        
    def get_batch(self, batch_ids):
        batch, labels = [], []
        for line_id in batch_ids:
            line = linecache.getline(bnc_word2vec,line_id).split('\t')
            if len(line)!=2: continue
            word_id,context_id = line
            batch.append(int(word_id))
            labels.append([int(context_id)])
        return np.array(batch),np.array(labels)
            
    def next_batch(self, n):
        if self.cursor+n > self.size:
            self.epoch += 1
            self.cursor = 0
            random.shuffle(self.data_ids)
        batch_ids = self.data_ids[self.cursor:self.cursor+n]
        self.cursor += n
        return self.get_batch(batch_ids)

In [5]:
t = DataIterator(data_ids)
tt = t.next_batch(10)
tt

(array([  110, 16648,  8663,  1989, 25158,   267,   125,  5378,   149]),
 array([[   285],
        [   246],
        [     4],
        [   828],
        [   158],
        [ 26355],
        [129769],
        [   281],
        [    58]]))

In [6]:
def reset_graph():
    if 'sess' in globals() and sess: sess.close()
    tf.reset_default_graph()
    
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/cpu:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 15935377572505960904, name: "/gpu:0"
 device_type: "GPU"
 memory_limit: 10498395341
 locality {
   bus_id: 1
 }
 incarnation: 16196948162637181691
 physical_device_desc: "device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:01:00.0"]

In [30]:
reset_graph()

target_size = len(cpt2i)
context_size = len(vr2i)
i2cpt = {i:cpt for cpt,i in cpt2i.iteritems()}

batch_size = 128
embedding_size = 128

# valid_size = 16     # Random set of words to evaluate similarity on.
# valid_window = 1000  # Only pick dev samples in the head of the distribution.
# valid_examples = np.random.choice(valid_window, valid_size, replace=False)
l = ['dog','cat','bird','animal',
     'book','house','kitchen','room',
     'pen','fork','government','news',
     'price','money','computer','school']
assert all(l_i in cpt2i for l_i in l)
valid_examples = [cpt2i[cpt] for cpt in l]
num_sampled = 64    # Number of negative examples to sample.

graph = tf.Graph()

with graph.as_default():

    # Input data.
    train_inputs = tf.placeholder(tf.int32, shape=[None])
    train_labels = tf.placeholder(tf.int32, shape=[None, 1])
    valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

    # Ops and variables pinned to the CPU because of missing GPU implementation
    with tf.device('/gpu:0'):
        # Look up embeddings for inputs.
        embeddings = tf.Variable(
            tf.random_uniform([target_size, embedding_size], -1.0, 1.0))
        embed = tf.nn.embedding_lookup(embeddings, train_inputs)

        # Construct the variables for the NCE loss
        nce_weights = tf.Variable(
            tf.truncated_normal([context_size, embedding_size],
                                stddev=1.0 / math.sqrt(embedding_size)))
        nce_biases = tf.Variable(tf.zeros([context_size]))

    # Compute the average NCE loss for the batch.
    # tf.nce_loss automatically draws a new sample of the negative labels each
    # time we evaluate the loss.
    loss = tf.reduce_mean(
        tf.nn.nce_loss(weights=nce_weights,
                       biases=nce_biases,
                       labels=train_labels,
                       inputs=embed,
                       num_sampled=num_sampled,
                       num_classes=context_size))

    # Construct the SGD optimizer using a learning rate of 1.0.
    optimizer = tf.train.AdamOptimizer().minimize(loss)

    # Compute the cosine similarity between minibatch examples and all embeddings.
    norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
    normalized_embeddings = embeddings / norm
    valid_embeddings = tf.nn.embedding_lookup(
        normalized_embeddings, valid_dataset)
    similarity = tf.matmul(
        valid_embeddings, normalized_embeddings, transpose_b=True)

    # Add variable initializer.
    init = tf.global_variables_initializer()

num_epochs = 1
num_steps = (int)((len(data_ids)/batch_size)*num_epochs)

with tf.Session(graph=graph) as session:
    # We must initialize all variables before we use them.
    init.run()
    print('Initialized')
    
    gen = DataIterator(data_ids)

    average_loss = 0
    for step in xrange(num_steps):
        batch_inputs, batch_labels = gen.next_batch(batch_size)
        feed_dict = {train_inputs: batch_inputs, train_labels: batch_labels}

        # We perform one update step by evaluating the optimizer op (including it
        # in the list of returned values for session.run()
        _, loss_val = session.run([optimizer, loss], feed_dict=feed_dict)
        average_loss += loss_val

        if step % 200 == 0:
            if step > 0:
                average_loss /= 2000
            # The average loss is an estimate of the loss over the last 2000 batches.
            print('Average loss at step ', step, ': ', average_loss)
            average_loss = 0

        # Note that this is expensive (~20% slowdown if computed every 500 steps)
        if step % 1000 == 0:
            sim = similarity.eval()
            for i in xrange(valid_size):
                valid_word = i2cpt[valid_examples[i]]
                top_k = 8  # number of nearest neighbors
                nearest = (-sim[i, :]).argsort()[1:top_k + 1]
                log_str = 'Nearest to %s:' % valid_word
                for k in xrange(top_k):
                    close_word = i2cpt[nearest[k]]
                    log_str = '%s %s,' % (log_str, close_word)
                print(log_str)
    final_embeddings = normalized_embeddings.eval()

In [29]:
'''
LAST RESULTS

Nearest to dog: horse, cat, rabbit, man, boy, pig, girl, animal,
Nearest to cat: dog, rabbit, horse, pig, character, lad, monster, boy,
Nearest to bird: animal, fish, specie, creature, rabbit, snake, insect, mammal,
Nearest to animal: bird, fish, plant, creature, predator, dog, horse, specie,
Nearest to book: poem, paper, volume, piece, man, kind, document, article,
Nearest to house: villa, pub, palace, mansion, garage, man, castle, building,
Nearest to kitchen: bathroom, bedroom, room, laboratory, garage, person, context, studio,
Nearest to room: bedroom, space, kitchen, cabin, hall, place, strategy, time,
Nearest to pen: paper, card, pencil, number, scheme, book, box, year,
Nearest to fork: knife, gun, spoon, conception, plank, grin, hook, elbow,
Nearest to government: administration, authority, regime, state, politician, parliament, king, time,
Nearest to news: idea, story, outcome, evidence, player, message, piece, sound,
Nearest to price: rate, value, cost, production, fee, number, wage, sale,
Nearest to money: cash, stuff, sum, game, time, kind, fund, sort,
Nearest to computer: machine, pc, system, microcomputer, database, printer, processor, question,
Nearest to school: university, hospital, church, college, village, library, factory, form,
('Average loss at step ', 7674200, ': ', 2.0442353295087816)
('Average loss at step ', 7674400, ': ', 2.2573078954219818)
('Average loss at step ', 7674600, ': ', 2.0706734832525253)

'''

0